In [ ]:
# 這LIHKG_SCRAPPING必需安裝Selenium和Firefox瀏覽器
!pip install selenium

In [5]:
# 1.拿取連登網站某帖子的html碼

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time

# Enable headless mode
options = Options()
options.headless = True

driver = webdriver.Firefox(options=options)
driver.get("https://lihkg.com/thread/3566438/page/1") #輸入你想爬取帖子的URL連結

time.sleep(2)
elem = driver.find_element("xpath", "//*")
source_code = elem.get_attribute("outerHTML")
print(source_code)

driver.quit()

<html lang="yue"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0,viewport-fit=cover"><meta name="apple-itunes-app" content="app-id=1178373396"><meta property="al:ios:app_store_id" content="880569525"><meta property="al:ios:app_name" content="LIHKG 討論區"><meta property="og:site_name" content="LIHKG 討論區"><title>胡啟敢/ 內山昂輝 走難去咗日本就評論香港政事，典型貪生怕死左翼行為 | LIHKG</title><link rel="stylesheet" href="https://cdn.lihkg.com/static/2311033837-fdbwr/css/styles.css"><link href="/manifest.json" rel="manifest"><link href="https://cdn.lihkg.com/assets/images/safari-pinned-tab.svg" color="#fbc308" rel="mask-icon"><link href="https://cdn.lihkg.com/assets/images/logo-256.png" rel="apple-touch-icon"><link href="https://cdn.lihkg.com/assets/web/style.css?ver=13032023" rel="stylesheet"><style>#loader{position:fixed;left:0;right:0;top:0;bottom:0;background:#fff;display:flex;flex-direction:column;justify-content:center;align-items:center;z

In [11]:
#2.從帖子的html碼拿取title

import re

html_title = source_code

# Use regular expressions to extract title content
pattern = r"<title>(.*?)</title>"
match = re.search(pattern, html_title)

if match:
    extracted_title = match.group(1)
    # 去除末尾的" | LIHKG"
    extracted_title = extracted_title.split(" | ")[0]
else:
    extracted_title = None

print(extracted_title)

工會冀市民接受的士加價　指疫情對司機收入影響很大


In [12]:
#3.從帖子的html碼拿取主要內容和youtube連結如果該帖子有youtube連結

import re
from bs4 import BeautifulSoup

def extract_text_from_content(content):
    # Extract Chinese text and punctuation marks
    pattern = r'(?<=[\u4e00-\u9fff\S])[\w\s\S]+(?=[\u4e00-\u9fff\S])'
    chinese_text = re.findall(pattern, content)
    return ' '.join(chinese_text)

def remove_links(text):
    # Remove link
    pattern = r'http\S+'
    text_without_links = re.sub(pattern, '', text)
    return text_without_links

def extract_content_and_youtube_link(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    # Extract useful content
    description_tag = soup.find('meta', {'name': 'description'})
    content = description_tag['content']
    content_text = BeautifulSoup(content, 'html.parser').text.strip()
    
    # Extract text
    extracted_text = extract_text_from_content(content_text)
    
    # Remove link
    extracted_text = remove_links(extracted_text)
    
    # Extract YouTube link
    youtube_link = 'None'
    a_tags = soup.find_all('a')
    for a_tag in a_tags:
        if 'youtu.be' in a_tag['href']:
            youtube_link = a_tag['href']
            break
    
    return extracted_text, youtube_link


html = source_code

content, youtube_link = extract_content_and_youtube_link(html)

print("Main_Content:", content)
print("YouTube_link:", youtube_link)

Main_Content: 政會議批准的士落旗加價3元，每次「跳錶」亦會相應加價，7月17日生效。

汽車交通運輸業總工會九龍分會副主任杜燊棠說，今次加價已經滯後了4年，而且加幅溫和，希望市民諒解和接受。

杜燊棠在本台節目《千禧年代》說，疫情對的士司機的收入影響很大，過去兩年雖然的士車主有減車租，但的士司機整體收入減少很多，最主要是由於政府打擊非法營運車輛不力，令的士客源減少了很多。他亦預計，的士車主會增加車租，希望車主不要太快加車租。

香港的士小巴商總會理事長周國強說，對上一次的士加價已經是5年前，現時的士車租只是疫情前的8折，預計日後會減少對司機的補貼，相信車租日後會調整至疫情前的水平，但不會在加價後馬上調整，會先觀察半個月至一個月。


YouTube_link: None


In [13]:
# 4.從帖子的html碼拿取所有評論

import csv
from bs4 import BeautifulSoup

def remove_links(text):
    pattern = r'http\S+'
    text_without_links = re.sub(pattern, '', text)
    return text_without_links

html_code = source_code

# 使用BeautifulSoup解析HTML
soup = BeautifulSoup(html_code, 'html.parser')

# Use CSS selector to select all matching tags
div_tags = soup.select('div[data-ast-root="true"]')

# Extract and filter content from duplicate tags
useful_contents = list(set(tag.text for tag in div_tags))


filtered_contents = [remove_links(content) for content in useful_contents]

# Combine content
combined_content = ', '.join(filtered_contents)

combined_content

'有無問D真係的士搵飯食司機??\n\n根本D的租車個班PK想加. \n跟手加租車費.\n\n真係的士搵飯食司機唔多想加.\n越加越少客., 其實呢d工會只係車主玩曬\n工會d頭目全部都係車主\n佢哋口中就話為左業界\n事實上搞咁多野都係想交租\n\n租車司機分分鐘仲比以前賺少左\n因為少左人答車，車租又貴左\n\n利申兼職司機, 加啦加到起錶1000 就差唔多，俾唔起錢咪撚搭的士, 市民冀工會接受的士減價\u3000指疫情對市民收入影響很大, 大家只要唔好坐的士，等佢無需求，佢就自然收皮㗎啦, 咁係因為人人為搵食轉行，你班DC9就日日響屋企等工開，要老婆養, 工會帶頭攬炒成個行業, 接受, 但唔搭係咪都唔得?, 痰惠豬就係的士狗大集團老闆 唔加點得, 有得唔接受架咩, 全香港得的士司機受影響？\n政府叫你同心抗疫啊\n你叫加價係咪想同政府唱反調, 無錯.支持的士直接加價到100, 我生活壓力都好大呀\n又要錢食飯\n又要供樓\n咁可唔可以比返啲mpf我, 全港DC9一年逃左幾多稅？, e家一個的士牌幾錢話？, 唔屌油價，唔屌車租，屌客?, 加完價, 車主加租, 又少咗人搭, 行政會議批准的士落旗加價3元，每次「跳錶」亦會相應加價，7月17日生效。\n\n汽車交通運輸業總工會九龍分會副主任杜燊棠說，今次加價已經滯後了4年，而且加幅溫和，希望市民諒解和接受。\n\n杜燊棠在本台節目《千禧年代》說，疫情對的士司機的收入影響很大，過去兩年雖然的士車主有減車租，但的士司機整體收入減少很多，最主要是由於政府打擊非法營運車輛不力，令的士客源減少了很多。他亦預計，的士車主會增加車租，希望車主不要太快加車租。\n\n香港的士小巴商總會理事長周國強說，對上一次的士加價已經是5年前，現時的士車租只是疫情前的8折，預計日後會減少對司機的補貼，相信車租日後會調整至疫情前的水平，但不會在加價後馬上調整，會先觀察半個月至一個月。\n\n,  加完又係收8折的\n車主賺到傻, 咁疫情後記得減返價, 市民無影響?, 接受，點解唔接受？\n\n BTW我都有三四年無搭過的士'

In [4]:
# 5.從帖子的html碼拿取該帖子的分類類型

import re


html_code = source_code

# Find the value of category_id using regular expression
pattern = r'category_id=(\d+)'
match = re.search(pattern, html_code)

# Extract the value of category_id
if match:
    category_id = match.group(1)
    print(f"The category_id is: {category_id}")
else:
    category_id = 'None'
    print("category_id not found.")

The category_id is: 5


In [15]:
# 6.將上面5個步驟所拿取的內容存入名為'data'的CSV FILE

import csv

# Define the data to be saved
data = [
    {'Title': extracted_title, 'Main_content': content, 'YoutubeURL': youtube_link, 'Comments': combined_content,
     'Category':category_id}]

# Define CSV file path
csv_file = 'data.csv'

# Write to CSV file
with open(csv_file, 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Main_content', 'YoutubeURL', 'Comments','Category'])
    writer.writeheader()
    writer.writerows(data)

print("Data successfully saved to CSV file")

Data successfully saved to CSV file


In [29]:
# 7.上面6個步驟都是只能拿取單一帖子的內容, 而以下這個function是一次過自動拿取某範圍的帖子並以DataFrame形式顯示範圍內帖子的所有內容

from selenium.webdriver.firefox.options import Options
import pandas as pd
import time
import re
from selenium import webdriver
from bs4 import BeautifulSoup


def scrape_posts(start_post, end_post):
    # 创建空的DataFrame
    df = pd.DataFrame(columns=['Post_number', 'Title', 'Main_content', 'Youtube_link', 'Comments', 'Category'])
    
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)
    
    for post_number in range(start_post, end_post + 1):
        post_url = f"https://lihkg.com/thread/{post_number}/page/1"
        
        driver.get(post_url)
        time.sleep(2)
        elem = driver.find_element("xpath", "//*")
        source_code = elem.get_attribute("outerHTML")
        
        pattern = r"<title>(.*?)</title>"
        match = re.search(pattern, source_code)
        if match:
            extracted_title = match.group(1).split(" | ")[0]
        else:
            extracted_title = None
        
        soup = BeautifulSoup(source_code, 'html.parser')
        description_tag = soup.find('meta', {'name': 'description'})
        content = description_tag['content']
        content_text = BeautifulSoup(content, 'html.parser').text.strip()
        extracted_text = re.sub(r'http\S+', '', content_text)
        
        youtube_link = 'None'
        a_tags = soup.find_all('a')
        for a_tag in a_tags:
            if 'youtu.be' in a_tag['href']:
                youtube_link = a_tag['href']
                break
        
        div_tags = soup.select('div[data-ast-root="true"]')
        useful_contents = list(set(tag.text for tag in div_tags))
        filtered_contents = [re.sub(r'http\S+', '', content) for content in useful_contents]
        combined_content = ', '.join(filtered_contents)
        
        pattern = r'category_id=(\d+)'
        match = re.search(pattern, source_code)
        if match:
            category_id = match.group(1)
        else:
            category_id = 'None'
        
        # Add post information to one row of DataFrame
        df = df.append({
            'Post_number': post_number,
            'Title': extracted_title,
            'Main_content': extracted_text,
            'Youtube_link': youtube_link,
            'Comments': combined_content,
            'Category': category_id
        }, ignore_index=True)
    
    driver.quit()
    return df


df = scrape_posts(3562857, 3562857+10) #輸入想拿取帖子的範圍
print(df)

C:\Users\yeung\AppData\Local\Temp\ipykernel_19736\2737491105.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\yeung\AppData\Local\Temp\ipykernel_19736\2737491105.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\yeung\AppData\Local\Temp\ipykernel_19736\2737491105.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\yeung\AppData\Local\Temp\ipykernel_19736\2737491105.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\yeung\AppData\Local\Temp\ipykernel_19736\2737491105.py:60: FutureWarning: The frame.append method is deprecated

   Post_number                                 Title  \
0      3562857        想問仲有咩名校係咁, 德望學校 中一至中三中文科以普通話授課   
1      3562858  學校淪謎之片拍攝場！男老師狂拍女生淫欲照　傳其他老師學生加入「共享資源」   
2      3562859              WHO稱中國已提供兒童肺炎數據，未檢測到新病原體   
3      3562860                       下個月尾上金手指，有冇人好期待   
4      3562861          沙特ETF周三在港掛牌 陳茂波：資產規模全球同類產品最大   
5      3562862                     愈唔識教仔嘅老豆老母平均生3-4個   
6      3562863                    返工唔開心咪撚忍，直接辭佢老尾臭西！   
7      3562864            ［有圖］我都覺得連登年青黃c人好有風濕唔會上大陸玩嘅   
8      3562865                       做乜依家咁多扮聾啞人士叫人捐錢   
9      3562866                咪撚再賭啦 其實你入幾多拎幾多 咪又係會輸返   
10     3562867                              唔知做咗咩又一日   

                                         Main_content  \
0                         想問仲有咩名校係咁\n陀地妹妹講普通話即刻CHEAP晒   
1   學校淪謎之片拍攝場！男老師狂拍女生淫欲照　傳其他老師學生加入「共享資源」 - 鏡週刊 Mir...   
2   世界衛生組織表示，中國已向其分享近來兒童呼吸道疾病激增的相關數據。此前一天，該機構稱正在尋求...   
3                            又可以睇到劉德華梁朝偉，好似見返好耐冇見既朋友咁   
4   財政司司長陳茂波說，過去一年逐步擺脫疫情困擾，但疫後復蘇進程存在一定挑戰和壓

In [30]:
# 8.將Category的值轉為對應的分類台

category_mapping = {
    '3': '政事台',
    '4': '手機台',
    '5': '時事台',
    '6': '體育台',
    '7': '娛樂台',
    '8': '動漫台',
    '9': 'Apps台',
    '10': '遊戲台',
    '11': '影視台',
    '12': '講故台',
    '13': '潮流台',
    '14': '上班台',
    '15': '財經台',
    '16': '飲食台',
    '17': '旅遊台',
    '18': '學術台',
    '19': '校園台',
    '20': '汽車台',
    '21': '音樂台',
    '22': '硬件台',
    '23': '攝影台',
    '24': '玩具台',
    '25': '寵物台',
    '26': '軟件台',
    '27': '活動台',
    '28': '站務台',
    '30': '感情台',
    '31': '創意台',
    '32': '黑洞',
    '33': '政事台',
    '34': '直播台',
    '35': '電訊台',
    '36': '健康台',
    '37': '房屋台',
    '38': 'World',
    '39': '家庭台',
    '40': '美容台',
    '41': '電器台'
}

df['Classification_table'] = df['Category'].map(category_mapping)

In [31]:
df

,Post_number,Title,Main_content,Youtube_link,Comments,Category,Classification_table
0,3562857,"想問仲有咩名校係咁, 德望學校 中一至中三中文科以普通話授課",想問仲有咩名校係咁\n陀地妹妹講普通話即刻CHEAP晒,None,"德望肉絲, 著校裙加多句「我草」, 好快你唔講蝗語就會俾人杯葛\n14年果陣係香討講大陸禁廣...",19,校園台
1,3562858,學校淪謎之片拍攝場！男老師狂拍女生淫欲照 傳其他老師學生加入「共享資源」,學校淪謎之片拍攝場！男老師狂拍女生淫欲照 傳其他老師學生加入「共享資源」 - 鏡週刊 Mir...,https://youtu.be/0EJ3TTjoZjs?embeds_referring_...,"你傳媒話「淫欲照」，但相都冇張睇吓，我點信你？, 關鍵詞？, 本來學生妹就係咁用架啦\n係而...",5,時事台
2,3562859,WHO稱中國已提供兒童肺炎數據，未檢測到新病原體,世界衛生組織表示，中國已向其分享近來兒童呼吸道疾病激增的相關數據。此前一天，該機構稱正在尋求...,None,"收到錢啦，冇事啦。, 數據呢？, 世界衛生組織表示，中國已向其分享近來兒童呼吸道疾病激增的相...",5,時事台
3,3562860,下個月尾上金手指，有冇人好期待,又可以睇到劉德華梁朝偉，好似見返好耐冇見既朋友咁,None,又可以睇到劉德華梁朝偉，好似見返好耐冇見既朋友咁,11,影視台
4,3562861,沙特ETF周三在港掛牌 陳茂波：資產規模全球同類產品最大,財政司司長陳茂波說，過去一年逐步擺脫疫情困擾，但疫後復蘇進程存在一定挑戰和壓力，必須加緊鞏固...,None,"而家都有美股ETF\n不過成交好低\n講真不如真接買美股好過 , 唔好忘記呢隻係沙特etf，...",15,財經台
5,3562862,愈唔識教仔嘅老豆老母平均生3-4個,點解嘅,None,"點解嘅, 恆哥 就嚟成為香港生育大使, 教育水平越低，或者越窮嘅人，會傾向生越多嘅子女，好似...",5,時事台
6,3562863,返工唔開心咪撚忍，直接辭佢老尾臭西！,靠 00 後打造更良好工作環境,None,"引入支国專才, 靠 00 後打造更良好工作環境 , 梗係唔自己走，等佢炒，唔炒就坐係度白𢭃人...",19,校園台
7,3562864,［有圖］我都覺得連登年青黃c人好有風濕唔會上大陸玩嘅,但係身體卻很誠實\n\n***蓮塘口岸條人龍超過1km啦DLLM,None,"溫馨提示:\n大角咀埃華街漢堡人食完痾L到大腸都必埋出嚟, 埃華街只係得一間漢堡\n你信唔信...",17,旅遊台
8,3562865,做乜依家咁多扮聾啞人士叫人捐錢,屌你老味食緊飯都可以冇啦啦行埋嚟面前揮手，張紙好似仲要寫捐100\n\n頭先等車又嚟多個，揮...,None,"十幾年前已經有啦\n賣公仔, 唔係講緊個手法好新喎，係多咗好多，同埋猖狂到行入餐廳叫人捐, ...",None,NaN
9,3562866,咪撚再賭啦 其實你入幾多拎幾多 咪又係會輸返,戒賭啦屌你,None,"十蚊注輸左都係輸支津路姐, 講得幾好, 有拎既 但入錢既一定多100倍, 轉做莊先有錢搵, ...",1,NaN


In [32]:
# 將上面創建的df存入名為'posts'的csv file
try:
    df.to_csv('posts.csv', index=False, encoding='utf-8-sig')
    print("CSV file saved successfully!")
except Exception as e:
    print(f"An error occurred while saving the CSV file:：{str(e)}")

CSV file saved successfully!
